# Data structures in C++

In [1]:
#include <iostream>
using namespace std;

## Trees

Trees are hierarchical data structures consisting of nodes connectedby edges. Each tree has root node from which all other nodes descend.

Nodes in a tree are connected in a particular way without forming and cycles. Trees are widely used for organizing and managing data efficiently.

### Binary Trees

This is a tree data structure in which each node has at most two childern, referred to as left child and right child.

- Each node can have zero, one or two children
- The orden of childern matters, the left child is usually considered smaller, and the right child is usually larger

![BT](https://miro.medium.com/v2/resize:fit:720/format:webp/0*cfgc3gvJ4cJiFB9G.png)

In [2]:
struct TreeNode {
    int data;
    TreeNode* left;
    TreeNode* right;

    TreeNode(int val) : data(val), left(nullptr), right(nullptr) {}
};


### Binary Search Tre
This is a tree data structure in which: 

- Each node's left child contains a value smaller that the node's value 

- Each node's right child contains a value greater that the node's value

This property enables efficient searching, insertion and deletion operations.

![BST](https://media.geeksforgeeks.org/wp-content/uploads/20220730030128/Screenshot20220730at30104AM-660x431.png)


In [3]:
class BinarySearchTree {
private:
    TreeNode* root;

    TreeNode* insert(TreeNode* root, int val) {
        if (root == nullptr) {
            return new TreeNode(val);
        }
        if (val < root->data) {
            root->left = insert(root->left, val);
        } else {
            root->right = insert(root->right, val);
        }
        return root;
    }

public:
    BinarySearchTree() : root(nullptr) {}

    void insert(int val) {
        root = insert(root, val);
    }
};


### AVL Trees

An AVL tree is a self-balancing binary search tree in which the heights of the two child subtrees of any node differ at most by one. 

AVL trees maintain a balance factor for each node, ensuring that the tree remains balanced after insertion and deletion.

![AVL](https://learnersbucket.com/wp-content/uploads/2021/03/AVL-Tree-in-Javascript.png)

In [4]:
class AVLTree {
private:
    struct AVLNode {
        int data;
        AVLNode* left;
        AVLNode* right;
        int height;

        AVLNode(int val) : data(val), left(nullptr), right(nullptr), height(1) {}
    };

    AVLNode* root;

    int getHeight(AVLNode* node) {
        if (node == nullptr) return 0;
        return node->height;
    }

    int getBalance(AVLNode* node) {
        if (node == nullptr) return 0;
        return getHeight(node->left) - getHeight(node->right);
    }

    AVLNode* rotateRight(AVLNode* y) {
        AVLNode* x = y->left;
        AVLNode* T2 = x->right;

        x->right = y;
        y->left = T2;

        y->height = 1 + max(getHeight(y->left), getHeight(y->right));
        x->height = 1 + max(getHeight(x->left), getHeight(x->right));

        return x;
    }

    AVLNode* rotateLeft(AVLNode* x) {
        AVLNode* y = x->right;
        AVLNode* T2 = y->left;

        y->left = x;
        x->right = T2;

        x->height = 1 + max(getHeight(x->left), getHeight(x->right));
        y->height = 1 + max(getHeight(y->left), getHeight(y->right));

        return y;
    }

    AVLNode* insert(AVLNode* node, int val) {
        if (node == nullptr) return new AVLNode(val);

        if (val < node->data) {
            node->left = insert(node->left, val);
        } else if (val > node->data) {
            node->right = insert(node->right, val);
        } else {
            return node; // Duplicate values not allowed
        }

        node->height = 1 + max(getHeight(node->left), getHeight(node->right));

        int balance = getBalance(node);

        // Left Left Case
        if (balance > 1 && val < node->left->data) {
            return rotateRight(node);
        }
        // Right Right Case
        if (balance < -1 && val > node->right->data) {
            return rotateLeft(node);
        }
        // Left Right Case
        if (balance > 1 && val > node->left->data) {
            node->left = rotateLeft(node->left);
            return rotateRight(node);
        }
        // Right Left Case
        if (balance < -1 && val < node->right->data) {
            node->right = rotateRight(node->right);
            return rotateLeft(node);
        }

        return node;
    }

public:
    AVLTree() : root(nullptr) {}

    void insert(int val) {
        root = insert(root, val);
    }
};


### Red_Black Trees

Red-Black Trees are another type of self-balancing binary search tree.

They maintain balance trough a set of color properties and rotations. 

- These trees ensure that the longest path from the root to any leaf is no more than twice the lenght of the shortest path

- Every node is eaither red or black

- The root and leaves are always black

- Red nodes cannot have red children

- Every path from a node to its descendant NULL nodes must contain the same number of black nodes.

![RBT](https://walkccc.me/CLRS/img/13.1-1-2.png)

In [5]:
enum Color { RED, BLACK };

struct RBTreeNode {
    int data;
    Color color;
    RBTreeNode* left;
    RBTreeNode* right;
    RBTreeNode* parent;

    RBTreeNode(int val) : data(val), color(RED), left(nullptr), right(nullptr), parent(nullptr) {}
};

class RedBlackTree {
private:
    RBTreeNode* root;

    void rotateLeft(RBTreeNode* node) {
        RBTreeNode* rightChild = node->right;
        node->right = rightChild->left;
        if (rightChild->left != nullptr) {
            rightChild->left->parent = node;
        }
        rightChild->parent = node->parent;
        if (node->parent == nullptr) {
            root = rightChild;
        } else if (node == node->parent->left) {
            node->parent->left = rightChild;
        } else {
            node->parent->right = rightChild;
        }
        rightChild->left = node;
        node->parent = rightChild;
    }

    void rotateRight(RBTreeNode* node) {
        RBTreeNode* leftChild = node->left;
        node->left = leftChild->right;
        if (leftChild->right != nullptr) {
            leftChild->right->parent = node;
        }
        leftChild->parent = node->parent;
        if (node->parent == nullptr) {
            root = leftChild;
        } else if (node == node->parent->left) {
            node->parent->left = leftChild;
        } else {
            node->parent->right = leftChild;
        }
        leftChild->right = node;
        node->parent = leftChild;
    }

    void fixViolation(RBTreeNode* newNode) {
        RBTreeNode* parent = nullptr;
        RBTreeNode* grandParent = nullptr;

        while (newNode != root && newNode->color != BLACK && newNode->parent->color == RED) {
            parent = newNode->parent;
            grandParent = newNode->parent->parent;

            // Case: Parent is left child of Grandparent
            if (parent == grandParent->left) {
                RBTreeNode* uncle = grandParent->right;

                // Case: Uncle is also red
                if (uncle != nullptr && uncle->color == RED) {
                    grandParent->color = RED;
                    parent->color = BLACK;
                    uncle->color = BLACK;
                    newNode = grandParent;
                } else {
                    // Case: newNode is right child of parent
                    if (newNode == parent->right) {
                        rotateLeft(parent);
                        newNode = parent;
                        parent = newNode->parent;
                    }
                    // Case: newNode is left child of parent
                    rotateRight(grandParent);
                    swap(parent->color, grandParent->color);
                    newNode = parent;
                }
            } else {  // Case: Parent is right child of Grandparent
                RBTreeNode* uncle = grandParent->left;

                // Case: Uncle is also red
                if (uncle != nullptr && uncle->color == RED) {
                    grandParent->color = RED;
                    parent->color = BLACK;
                    uncle->color = BLACK;
                    newNode = grandParent;
                } else {
                    // Case: newNode is left child of parent
                    if (newNode == parent->left) {
                        rotateRight(parent);
                        newNode = parent;
                        parent = newNode->parent;
                    }
                    // Case: newNode is right child of parent
                    rotateLeft(grandParent);
                    swap(parent->color, grandParent->color);
                    newNode = parent;
                }
            }
        }
        root->color = BLACK;
    }

public:
    RedBlackTree() : root(nullptr) {}

    void insert(int val) {
        RBTreeNode* newNode = new RBTreeNode(val);
        root = insertHelper(root, newNode);
        fixViolation(newNode);
    }

    RBTreeNode* insertHelper(RBTreeNode* root, RBTreeNode* newNode) {
        if (root == nullptr) {
            return newNode;
        }

        if (newNode->data < root->data) {
            root->left = insertHelper(root->left, newNode);
            root->left->parent = root;
        } else if (newNode->data > root->data) {
            root->right = insertHelper(root->right, newNode);
            root->right->parent = root;
        }

        return root;
    }
};


### B-Trees

These are self-balancing tree data structures commonly used in databases and files systems. They are designed to work well with storage systems that read and write relatively large blocks of data. 

B-Trees maintain data in sorted order,  and allow searches, sequential access, insertions and deletions in logarithmic time.

![BTREE](https://en.algorithmica.org/hpc/data-structures/img/b-tree.jpg)
 

In [6]:
const int MIN_KEYS = 1;
const int MAX_KEYS = 2 * MIN_KEYS;

struct BTreeNode {
    int numKeys;
    int keys[MAX_KEYS];
    BTreeNode* children[MAX_KEYS + 1];
    bool isLeaf;

    BTreeNode(bool leaf) : numKeys(0), isLeaf(leaf) {
        for (int i = 0; i <= MAX_KEYS; ++i) {
            children[i] = nullptr;
        }
    }
};

class BTree {
private:
    BTreeNode* root;

    void splitChild(BTreeNode* parent, int index) {
        BTreeNode* child = parent->children[index];
        BTreeNode* newChild = new BTreeNode(child->isLeaf);
        newChild->numKeys = MIN_KEYS;

        for (int i = 0; i < MIN_KEYS; ++i) {
            newChild->keys[i] = child->keys[i + MIN_KEYS];
        }

        if (!child->isLeaf) {
            for (int i = 0; i < MIN_KEYS + 1; ++i) {
                newChild->children[i] = child->children[i + MIN_KEYS];
            }
        }

        child->numKeys = MIN_KEYS;

        for (int i = parent->numKeys; i >= index + 1; --i) {
            parent->children[i + 1] = parent->children[i];
        }

        parent->children[index + 1] = newChild;

        for (int i = parent->numKeys - 1; i >= index; --i) {
            parent->keys[i + 1] = parent->keys[i];
        }

        parent->keys[index] = child->keys[MIN_KEYS];

        parent->numKeys++;
    }

    void insertNonFull(BTreeNode* node, int key) {
        int i = node->numKeys - 1;

        if (node->isLeaf) {
            while (i >= 0 && key < node->keys[i]) {
                node->keys[i + 1] = node->keys[i];
                i--;
            }
            node->keys[i + 1] = key;
            node->numKeys++;
        } else {
            while (i >= 0 && key < node->keys[i]) {
                i--;
            }

            if (node->children[i + 1]->numKeys == MAX_KEYS) {
                splitChild(node, i + 1);

                if (key > node->keys[i + 1]) {
                    i++;
                }
            }
            insertNonFull(node->children[i + 1], key);
        }
    }

public:
    BTree() : root(nullptr) {}

    void insert(int key) {
        if (root == nullptr) {
            root = new BTreeNode(true);
            root->keys[0] = key;
            root->numKeys = 1;
        } else {
            if (root->numKeys == MAX_KEYS) {
                BTreeNode* newRoot = new BTreeNode(false);
                newRoot->children[0] = root;
                splitChild(newRoot, 0);
                int i = 0;
                if (newRoot->keys[0] < key) {
                    i++;
                }
                insertNonFull(newRoot->children[i], key);
                root = newRoot;
            } else {
                insertNonFull(root, key);
            }
        }
    }
};
